In [37]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

# Sistema de ecuaciones de Friedmann
def friedmann_system(t, y, Omega_m0, Omega_r0, Omega_Lambda0, Omega_k0):
    a, v = y
    da_dt = v
    dv_dt = -0.5 * (Omega_m0 / a**2 + 2 * Omega_r0 / a**3 ) + Omega_Lambda0 * a 
    return [da_dt, dv_dt]

# Función para resolver y graficar
def plot_friedmann(Omega_m0=0.3, Omega_r0=0.0, Omega_Lambda0=0.7, curvatura=0):
    # Calcular Omega_k0 a partir de la curvatura
    Omega_k0 = 1 - Omega_m0 - Omega_r0 - Omega_Lambda0
    if curvatura == -1:
        Omega_k0 = abs(Omega_k0)  # Universo abierto
    elif curvatura == 1:
        Omega_k0 = -abs(Omega_k0)   # Universo cerrado
    else:
        Omega_k0 = 0               # Universo plano
    
    # Condiciones iniciales
    a_initial = 1e-2  # Factor de escala inicial (muy pequeño para universo temprano)
    v_initial = np.sqrt((Omega_m0 / a_initial + Omega_r0 / a_initial**2 + Omega_Lambda0 * a_initial**2 + Omega_k0))  # Velocidad inicial
    t0 = 0
    t_final = 10  # Tiempo final adimensional
    
    # Resolución numérica
    sol = solve_ivp(
        friedmann_system,
        [t0, t_final],
        [a_initial, v_initial],
        args=(Omega_m0, Omega_r0, Omega_Lambda0, Omega_k0),
        t_eval=np.linspace(t0, t_final, 1000),
        method='RK45',
        rtol=1e-8,
        atol=1e-10
    )
    
    # Extraer resultados
    a = sol.y[0]
    da_dt = sol.y[1]  # Velocidad de expansión da/dt
    t = sol.t

    
    # Graficar
    fig, axs = plt.subplots(2, 1, figsize=(10, 15), sharex=True)
    
    # Gráfico de a(t)
    axs[0].plot(t, a, label=f"$\Omega_m={Omega_m0}$, $\Omega_r={Omega_r0}$, $\Omega_\Lambda={Omega_Lambda0}$, $\Omega_k={Omega_k0:.3f}$")
    axs[0].set_ylabel("Factor de escala $a(t)$")
    axs[0].set_title("Evolución del factor de escala $a(t)$")
    axs[0].set_xlabel("Tiempo adimensional $\widetilde{t}$")
    axs[0].legend()
    axs[0].grid()
   
    # Gráfico de da/dt
    axs[1].plot(t, da_dt, label="Velocidad de expansión $da/dt$")
    axs[1].set_ylabel("Velocidad $da/dt$")
    axs[1].set_title("Evolución de la velocidad de expansión $da/dt$")
    axs[1].legend()
    axs[1].set_xlabel("Tiempo adimensional $\widetilde{t}$")
    axs[1].grid()
    
    plt.tight_layout()
    plt.show()

# Interactividad con ipywidgets
interact(
    plot_friedmann,
    Omega_m0=FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description="$\Omega_m$"),
    Omega_r0=FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description="$\Omega_r$"),
    Omega_Lambda0=FloatSlider(value=0.7, min=-1.0, max=1.0, step=0.01, description="$\Omega_\Lambda$"),
    curvatura=Dropdown(options=[("Plano (k=0)", 0), ("Cerrado (k=1)", 1), ("Abierto (k=-1)", -1)], description="Curvatura")
)


interactive(children=(FloatSlider(value=0.3, description='$\\Omega_m$', max=1.0, step=0.01), FloatSlider(value…

<function __main__.plot_friedmann(Omega_m0=0.3, Omega_r0=0.0, Omega_Lambda0=0.7, curvatura=0)>